In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [ ]:
df=pd.read_csv('mashroom.csv')
df


In [ ]:
df.info()


In [ ]:
df.isnull().sum()


In [ ]:
df.duplicated().sum()


In [ ]:
for i in df.columns:
    plt.figure()
    sns.countplot(df[i])
    sns.set(style='whitegrid')


In [ ]:
df.columns


In [ ]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()


In [ ]:
for i in df.columns:
    df[i]=lb.fit_transform(df[i])
    df


In [ ]:
df


In [ ]:
corr=df.corr()


In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(data=corr,annot=True)


In [ ]:
df.drop('veil-type',axis=1,inplace=True)


In [ ]:
df.drop('cap-surface',axis=1,inplace=True)


In [ ]:
df.head()


In [ ]:
x=df.drop('class',axis=1)
x


In [ ]:
y=df['class']
y


In [ ]:
y.value_counts()


In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smt=SMOTE()
trainx,trainy=smt.fit_resample(x,y)


In [ ]:
trainy.value_counts()


In [ ]:
trainy


In [ ]:
from sklearn.model_selection  import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score


In [ ]:
lg=LogisticRegression()


In [ ]:
for i in range(1,100):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=i)
    lg.fit(x_train,y_train)
    pred_test=lg.predict(x_test)
    pred_train=lg.predict(x_train)
    if round(accuracy_score(y_test,pred_test)*100,1)==round(accuracy_score(y_train,pred_train)*100,1):
        print('Random state',i,'score is well')
        print('test score',accuracy_score(y_test,pred_test)*100)
        print('train score',accuracy_score(y_train,pred_train)*100)

In [ ]:
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV


In [ ]:
svc=SVC()
dtc=DecisionTreeClassifier()
knn=KNeighborsClassifier()
nb=MultinomialNB()


In [ ]:
param={'kernel':['linear','rbf','poly','sigmoid']}
gd=GridSearchCV(svc,param,cv=5)
gd.fit(x,y)
print(gd.best_params_)


In [ ]:
param={'criterion':['ginny','entropy'],'splitter':['best','random'],'max_features':['none','auto','sqrt','log2']}
gd=GridSearchCV(dtc,param)
gd.fit(x,y)
print(gd.best_params_)


In [ ]:
param={'n_neighbors':[7,5,11],'weights':['uniform','distance'],'algorithm':['auto','bell_tree','kd_tree','brute']}
gd=GridSearchCV(knn,param)
gd.fit(x,y)
print(gd.best_params_)


In [ ]:
svc=SVC(kernel='linear')
dtc=DecisionTreeClassifier(criterion='entropy',max_features='log2',splitter='random')
knn=KNeighborsClassifier(n_neighbors=5,weights='distance',algorithm='auto')
nb=MultinomialNB()


In [ ]:
def fun(f):
    f.fit(x_train,y_train)
    pred=f.predict(x_test)
    print('Accuracy Score',accuracy_score(y_test,pred))
    print('Confusion Matrix\n',confusion_matrix(y_test,pred))
    print('Classification Report\n',classification_report(y_test,pred))
    print('F1 score',f1_score(y_test,pred))


In [ ]:
fun(svc)


In [ ]:
fun(dtc)


In [ ]:
fun(knn)


In [ ]:
fun(nb)


In [ ]:
fun(lg)


Cross Validation


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


In [ ]:
score=cross_val_score(SVC(kernel='linear'),x,y,cv=5)
print(score)
print(score.mean())


In [ ]:
score=cross_val_score(DecisionTreeClassifier(criterion='entropy',max_features='log2',splitter='random'),x,y,cv=5)
print(score)
print(score.mean())


In [ ]:
score=cross_val_score(KNeighborsClassifier(n_neighbors=5,weights='distance',algorithm='auto'),x,y,cv=5)
print(score)
print(score.mean())


In [ ]:
score=cross_val_score(MultinomialNB(),x,y,cv=5)
print(score)
print(score.mean())


Ensemble Method


In [ ]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier


In [ ]:
rf=RandomForestClassifier(n_estimators=100,random_state=23)
ad=AdaBoostClassifier()
Gd=GradientBoostingClassifier(n_estimators=200,learning_rate=0.1)


In [ ]:
fun(rf)


In [ ]:
fun(ad)


In [ ]:
fun(gd)

Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier


In [ ]:
estimator=[]

estimator.append(('svc',SVC(kernel='linear')))
estimator.append(('dtc',DecisionTreeClassifier(criterion='entropy',max_features='log2',splitter='random')))
estimator.append(('knn',KNeighborsClassifier(n_neighbors=5,weights='distance',algorithm='auto')))
estimator.append(('nb',MultinomialNB()))


In [ ]:
vot_soft=VotingClassifier(estimators=estimator,voting='hard')
vot_soft.fit(x_train,y_train)
y_pred=vot_soft.predict(x_test)
print(y_pred)
#usint accuracy score
score=(accuracy_score(y_test,y_pred))
print('soft voting score % d'% score)


AUC ROC CURVE


In [ ]:
from sklearn.metrics import roc_curve


In [ ]:
y_pred_prob=rf.predict_proba(x_test)[:,1]


In [ ]:
y_pred_prob

In [ ]:
fpr,tpr,thersholds=roc_curve(y_test,y_pred_prob)


In [ ]:
fpr

In [ ]:
tpr

In [ ]:
thersholds


In [ ]:
plt.plot(fpr,tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Logistic Regression')
plt.show()


In [ ]:
import pickle
filename='Mashroom_Prediction.pkl'
pickle.dump(rf,open(filename,'wb'))

In [ ]:
res=pd.DataFrame()
res['Actual']=y_test
res['Predict']=rf.predict(x_test)

print(res)
sns.catplot(data=res)
